In [1]:
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
#get_ipython().run_line_magic('matplotlib', 'inline')
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.pipeline import Pipeline
import seaborn as sns
import pickle
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

import pandas as pd
import numpy as np
import re

Using TensorFlow backend.


In [2]:
f = open('nlp_train.json') 
import pandas as pd
import json
data = json.load(f) 

    # Iterating through the json 
    # list 
    #print(data["fkrr36o"]["emotion"])
f.close() 
fl=[]
    #print(data["fkrr36o"])
for u,v in data.items():
    l=[]
    l.append(u)
    l.append(v["body"])
    l.append(1 if v["emotion"]["anger"]==True else 0)
    l.append(1 if v["emotion"]["anticipation"]==True else 0)
    l.append(1 if v["emotion"]["disgust"]==True else 0)
    l.append(1 if v["emotion"]["fear"]==True else 0)
    l.append(1 if v["emotion"]["joy"]==True else 0)
    l.append(1 if v["emotion"]["love"]==True else 0)
    l.append(1 if v["emotion"]["optimism"]==True else 0)
    l.append(1 if v["emotion"]["pessimism"]==True else 0)
    l.append(1 if v["emotion"]["sadness"]==True else 0)
    l.append(1 if v["emotion"]["surprise"]==True else 0)
    l.append(1 if v["emotion"]["trust"]==True else 0)
    l.append(1 if v["emotion"]["neutral"]==True else 0)
    fl.append(l)

df = pd.DataFrame(fl, columns=["id","body","anger","anticipation","disgust","fear","joy","love","optimism","pessimism","sadness","surprise","trust","neutral"])
df.to_pickle("./dataframe.pkl")
print("Data loaded to the dataframe")

Data loaded to the dataframe


In [3]:
df = pd.read_pickle("./dataframe.pkl")
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
porter = PorterStemmer()
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)


In [4]:
def clean(data):
    data = data.lower()
    data = re.sub('<[^<]+?>', '', data)
    data=re.sub('[!#?,.:";]', '', data)
    data=re.sub(r'[0-9]+', '', data)
    data= re.sub("i'm","i am",data)
    data = stemSentence(data)
    return data

In [5]:
print("Preprocessing started")
df['body'] = df['body'].map(lambda x : clean(x))
print("Preprocessing completed")


Preprocessing started
Preprocessing completed


In [6]:


emotions = ["anger","anticipation","disgust","fear","joy","love","optimism","pessimism","sadness","surprise","trust","neutral"]
train, test= train_test_split(df, random_state=40, test_size=0.25, shuffle=True)

X_train= train['body']
X_test= test['body']
y_train=train.iloc[:,2:]
y_test=test.iloc[:,2:]
print(train.shape)
print(y_train.shape)
print(y_train.shape)
print(y_train.shape)
print(y_train.shape)
print(X_test.shape)

(1119, 14)
(1119, 12)
(1119, 12)
(1119, 12)
(1119, 12)
(374,)


In [7]:
def prep(y1_train):
    Y1_train=[]
    
    for i in y1_train:
        if i==0:
            Y1_train.append([1,0])
        else:
            Y1_train.append([0,1])
    return Y1_train


In [8]:
y1_train = prep(train["anger"].values)
y1_test =  prep(train["anger"].values)


y2_train = prep(train["anticipation"].values)
y2_test =  prep(train["anticipation"].values)

y3_train = prep(train["disgust"].values)
y3_test =  prep(train["disgust"].values)

y4_train = prep(train["fear"].values)
y4_test =  prep(train["fear"].values)

y5_train = prep(train["joy"].values)
y5_test =  prep(train[["joy"]].values)

y6_train = prep(train["love"].values)
y6_test =  prep(train["love"].values)

y7_train = prep(train["optimism"].values)
y7_test =  prep(train["optimism"].values)

y8_train = prep(train["pessimism"].values)
y8_test =  prep(train["pessimism"].values)

y9_train = prep(train["sadness"].values)
y9_test =  prep(train["sadness"].values)

y10_train = prep(train["surprise"].values)
y10_test =  prep(train["surprise"].values)

y11_train = prep(train["trust"].values)
y11_test =  prep(train["trust"].values)

y12_train = prep(train["neutral"].values)
y12_test =  prep(train["neutral"].values)



In [9]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

In [10]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [11]:
glove_file = open('glove.6B.100d.txt', encoding="utf8")
embeddings_dictionary={}
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = np.zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [12]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))        

In [24]:
input_1 = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(input_1)
LSTM_Layer1 = LSTM(128)(embedding_layer)
#LSTM_Layer2 = LSTM(128)(LSTM_Layer1)

output1 = Dense(2, activation='sigmoid')(LSTM_Layer1)
output2 = Dense(2, activation='sigmoid')(LSTM_Layer1)
output3 = Dense(2, activation='sigmoid')(LSTM_Layer1)
output4 = Dense(2, activation='sigmoid')(LSTM_Layer1)
output5 = Dense(2, activation='sigmoid')(LSTM_Layer1)
output6 = Dense(2, activation='sigmoid')(LSTM_Layer1)
output7 = Dense(2, activation='sigmoid')(LSTM_Layer1)
output8= Dense(2, activation='sigmoid')(LSTM_Layer1)
output9 = Dense(2, activation='sigmoid')(LSTM_Layer1)
output10 = Dense(2, activation='sigmoid')(LSTM_Layer1)
output11 = Dense(2, activation='sigmoid')(LSTM_Layer1)
output12 = Dense(2, activation='sigmoid')(LSTM_Layer1)


model = Model(inputs=input_1, outputs=[output1, output2, output3, output4, output5, output6, output7, output8,output9, output10, output11, output12])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])

In [21]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 200, 100)     1980600     input_4[0][0]                    
__________________________________________________________________________________________________
lstm_5 (LSTM)                   (None, 128)          117248      embedding_4[0][0]                
__________________________________________________________________________________________________
dense_14 (Dense)                (None, 2)            258         lstm_5[0][0]                     
__________________________________________________________________________________________________
dense_15 (

In [22]:
print(X_train.shape)
print(y_train.shape)

(1119, 200)
(1119, 12)


In [23]:
history = model.fit(x=X_train, y=[y1_train, y2_train, y3_train, y4_train, y5_train, y6_train, y7_train,y8_train,y9_train,y10_train,y11_train,y12_train], batch_size=128, epochs=30, verbose=1, validation_split=0.2)

Train on 895 samples, validate on 224 samples
Epoch 1/30
895/895 [==============================] - 11s 12ms/step - loss: 7.9649 - dense_14_loss: 0.7040 - dense_15_loss: 0.6912 - dense_16_loss: 0.6948 - dense_17_loss: 0.6968 - dense_18_loss: 0.6076 - dense_19_loss: 0.6533 - dense_20_loss: 0.6705 - dense_21_loss: 0.6932 - dense_22_loss: 0.6891 - dense_23_loss: 0.5973 - dense_24_loss: 0.6272 - dense_25_loss: 0.6399 - dense_14_f1: 0.5468 - dense_15_f1: 0.5669 - dense_16_f1: 0.5149 - dense_17_f1: 0.6125 - dense_18_f1: 0.8099 - dense_19_f1: 0.6710 - dense_20_f1: 0.6808 - dense_21_f1: 0.4408 - dense_22_f1: 0.6739 - dense_23_f1: 0.8186 - dense_24_f1: 0.7009 - dense_25_f1: 0.7621 - val_loss: 7.3134 - val_dense_14_loss: 0.6760 - val_dense_15_loss: 0.7044 - val_dense_16_loss: 0.6935 - val_dense_17_loss: 0.6918 - val_dense_18_loss: 0.4686 - val_dense_19_loss: 0.5278 - val_dense_20_loss: 0.6662 - val_dense_21_loss: 0.7033 - val_dense_22_loss: 0.6554 - val_dense_23_loss: 0.4742 - val_dense_24_loss:

In [28]:
emotion=["anger","anticipation","disgust","fear","joy","love","optimism","pessimism","sadness","surprise","trust","neutral"]
prediction = model.predict(X_test)
import sklearn
for j in range(12):
    anger=[]
    for i in range(len(prediction[1])):
        if prediction[j][i][0]>prediction[j][i][1]:
            anger.append(0)
        else:
            anger.append(1)
    #print(prediction[0]) 
    
    print("accuracy score of "+emotion[j]+"   : "+str(sklearn.metrics.accuracy_score(test[emotion[j]],anger)))


accuracy score of anger   : 0.4893048128342246
accuracy score of anticipation   : 0.4304812834224599
accuracy score of disgust   : 0.49732620320855614
accuracy score of fear   : 0.44919786096256686
accuracy score of joy   : 0.3074866310160428
accuracy score of love   : 0.4679144385026738
accuracy score of optimism   : 0.39572192513368987
accuracy score of pessimism   : 0.43315508021390375
accuracy score of sadness   : 0.35294117647058826
accuracy score of surprise   : 0.4679144385026738
accuracy score of trust   : 0.446524064171123
accuracy score of neutral   : 0.4037433155080214


In [29]:
emotion=["anger","anticipation","disgust","fear","joy","love","optimism","pessimism","sadness","surprise","trust","neutral"]
prediction = model.predict(X_test)

for j in range(12):
    anger=[]
    for i in range(len(prediction[1])):
        if prediction[j][i][0]>prediction[j][i][1]:
            anger.append(0)
        else:
            anger.append(1)
    #print(prediction[0]) 
    
    print("f1 score of "+emotion[j]+"   : "+str(sklearn.metrics.f1_score(test[emotion[j]],anger)))


f1 score of anger   : 0.4589235127478753
f1 score of anticipation   : 0.20817843866171
f1 score of disgust   : 0.5982905982905983
f1 score of fear   : 0.47715736040609136
f1 score of joy   : 0.26210826210826216
f1 score of love   : 0.17427385892116184
f1 score of optimism   : 0.5291666666666666
f1 score of pessimism   : 0.152
f1 score of sadness   : 0.45739910313901344
f1 score of surprise   : 0.18775510204081633
f1 score of trust   : 0.23048327137546468
f1 score of neutral   : 0.21754385964912284


In [30]:
import pickle
pkl_filename = "neuralmodel.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)